In [3]:
pip install 'praatio<5'

Defaulting to user installation because normal site-packages is not writeable
  Using cached praatio-4.4.0-py2.py3-none-any.whl (62 kB)

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: /apps/z_install_tree/linux-rocky8-ivybridge/gcc-12.2.0/python-3.10.8-pmtwsrrmcmrs6olvgx5xhepgh7gl5vro/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install pydub

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: /apps/z_install_tree/linux-rocky8-ivybridge/gcc-12.2.0/python-3.10.8-pmtwsrrmcmrs6olvgx5xhepgh7gl5vro/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Based on "https://www.khiajohnson.com/post/2021/textgrid-sqlite-database/"

In [3]:
from os.path import join
from praatio import tgio
import praatio
from pydub import AudioSegment
import pandas as pd
import os
import re


/home/z5313567/.local/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [9]:
print("\n------> Loading directories and files... ------\n")
tg_dir = '/srv/scratch/chacmod/auskidtalk'
print(f'TextGrid directory is at: {tg_dir}')

save_dir = '/srv/scratch/chacmod/auskidtalk_audio'
print(f'Split .wav files are stored at: {save_dir}')

save_df_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_dataframe.csv'
print(f'Dataframe file is stored at: {save_df_fp}')


------> Loading directories and files... ------

TextGrid directory is at: /srv/scratch/chacmod/auskidtalk
Split .wav files are stored at: /srv/scratch/chacmod/auskidtalk_audio
Dataframe file is stored at: /srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_dataframe.csv


In [10]:
print("\n------> Loading TextGrid files... ------\n")
tg_file_names = [file for file in os.listdir(tg_dir) if file.endswith('TextGrid')]
tg_files = [tg_dir+'/'+file_name for file_name in tg_file_names]

tg_files.remove('/srv/scratch/chacmod/auskidtalk/398_task1_kaldi.TextGrid')

print(f'The number of TextGrid files is {len(tg_files)}')
print('TextGrid files include:\n')
for tg_file in tg_files:
    print(tg_file)


------> Loading TextGrid files... ------

The number of TextGrid files is 79
TextGrid files include:

/srv/scratch/chacmod/auskidtalk/349_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/259_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/97_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/251_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/819_task1_GT.TextGrid
/srv/scratch/chacmod/auskidtalk/260_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/323_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/389_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/130_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/211_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/310_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/220_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/301_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/219_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/237_task1_kaldi.TextGrid
/srv/scratch/chacmod/auskidtalk/318_task1_kald

In [47]:
print("\n------> Extracting speeches from TextGrid files... ------\n")
#count0 = 0
extracted_audio = []
extracted_transcription = []
extracted_speakerID = []
extracted_filepath = []
extracted_duration = []
for tg_file in tg_files:
    # extract file_ID                                     '/srv/scratch/chacmod/auskidtalk/172_task1_kaldi.TextGrid'
    file_basename = os.path.basename(tg_file)             # 172_task1_kaldi.TextGrid 
    file_ID = os.path.splitext(file_basename)[0]          # 172_task1_kaldi
    split_by_underline = file_ID.split('_')               # ['172','task1', 'kaldi']
    speaker_ID = '_'.join(split_by_underline[:2])          # 172_task1
    
    # generate corresponding .wav.file
    # if TextGrid filename is '172_task1_kaldi.TextGrid', then wav filename should be '172_task1.wav'
    wav_file = os.path.join(tg_dir, speaker_ID+'.wav')
    audio = AudioSegment.from_wav(wav_file)
    
    tg = tgio.openTextgrid(tg_file)
    tier_names = tg.tierNameList
    
    print('Currently Processing:')
    print('TextGrid File:', tg_file)
    print('Tier names:', tier_names)
    print('\n')
    
    #count1 = 0
    
    # tier_names[1] contains the speaker
    # tier_names[2] contrins the hu-wrd interval or Final-wrd interval
    count = 1
    for xmin, xmax, text in tg.tierDict[tier_names[2]].entryList:
        if text != 'sil':
            split_audio = audio[xmin*1000:xmax*1000] # the unit is in milliseconds
            save_filepath = os.path.join(save_dir, speaker_ID+'_'+str(count)+'.wav')
            split_audio.export(save_filepath, format='wav')
            
            chars_to_ignore = '[\,\?\.\!\-\;\_\:\"]'
            text = re.sub(chars_to_ignore, ' ', text).lower()
            
            extracted_audio.append(split_audio)
            extracted_transcription.append(text)
            extracted_speakerID.append(speaker_ID)
            extracted_filepath.append(save_filepath)
            extracted_duration.append(xmax-xmin)
            
            count = count+1
'''
        count1 = count1 + 1
        if count1 == 20:
            break
    count0 = count0 + 1
    if count0 == 1:
        break
    
print('\n')
print(extracted_audio)
print(extracted_transcription)
print(extracted_speakerID)
print(extracted_filepath)
print(extracted_duration)
'''


------> Extracting speeches from TextGrid files... ------

Currently Processing:
TextGrid File: /srv/scratch/chacmod/auskidtalk/349_task1_kaldi.TextGrid
Tier names: ['Prompt', 'speaker-1', 'hu-wrd', 'flag']


Currently Processing:
TextGrid File: /srv/scratch/chacmod/auskidtalk/259_task1_kaldi.TextGrid
Tier names: ['Prompt', 'speaker-1', 'hu-wrd', 'flag']


Currently Processing:
TextGrid File: /srv/scratch/chacmod/auskidtalk/97_task1_kaldi.TextGrid
Tier names: ['Prompt', 'speaker-1', 'hu-wrd', 'flag']


Currently Processing:
TextGrid File: /srv/scratch/chacmod/auskidtalk/251_task1_kaldi.TextGrid
Tier names: ['Prompt', 'speaker-0', 'hu-wrd', 'flag']


Currently Processing:
TextGrid File: /srv/scratch/chacmod/auskidtalk/819_task1_GT.TextGrid
Tier names: ['Prompt', 'Nemo-sp-0', 'Final-wrd', 'Final-flag-nemo']


Currently Processing:
TextGrid File: /srv/scratch/chacmod/auskidtalk/260_task1_kaldi.TextGrid
Tier names: ['Prompt', 'speaker-1', 'hu-wrd', 'flag']


Currently Processing:
TextGrid

"\n        count1 = count1 + 1\n        if count1 == 20:\n            break\n    count0 = count0 + 1\n    if count0 == 1:\n        break\n    \nprint('\n')\nprint(extracted_audio)\nprint(extracted_transcription)\nprint(extracted_speakerID)\nprint(extracted_filepath)\nprint(extracted_duration)\n"

In [49]:
print("\n------> Generating dataframe ... ------\n")
df = pd.DataFrame(
        {'filepath': extracted_filepath,
         'duration': extracted_duration,
         'speaker_id': extracted_speakerID,
         'transcription': extracted_transcription,
         })

print("\n------> Saving dataframe to csv file... ------\n")
df.to_csv(save_df_fp, index=False)
print('Successfully saved dataframe to csv file at: ', save_df_fp)
print("Total number of speakers:", len(tg_files))
print("Total hours:", df['duration'].sum()/(60*60))


------> Generating dataframe ... ------


------> Saving dataframe to csv file... ------

Successfully saved dataframe to csv file at:  /srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_dataframe.csv
Total number of speakers: 79
Total hours: 2.3509678730813808


In [39]:
for tg_file in tg_files:
    if tg_file != "/srv/scratch/chacmod/auskidtalk/398_task1_kaldi.TextGrid":
        tg = tgio.openTextgrid(tg_file)  # Give it a file name, get back a Textgrid object
        tg_dir = '/srv/scratch/chacmod/auskidtalk'
        #print(tg_file)
        print(tg.tierNameList)

['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-0', 'hu-wrd', 'flag']
['Prompt', 'Nemo-sp-0', 'Final-wrd', 'Final-flag-nemo']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-0', 'hu-wrd', 'flag']
['Prompt', 'speaker-0', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'Nemo-sp-0', 'Final-wrd', 'Final-flag-nemo']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'speaker-1', 'hu-wrd', 'flag']
['Prompt', 'Nemo-sp-1', 'Final-wrd', 'Final-flag-nemo']
['Prompt', 'Nemo-sp-0', 'Final-wrd

In [35]:
tg = tgio.openTextgrid("/srv/scratch/chacmod/auskidtalk/398_task1_kaldi_2.TextGrid")
print(tg_file)
print(tg.tierNameList)

/srv/scratch/chacmod/auskidtalk/339_task1_kaldi.TextGrid
['speaker_0-kaldi-words', 'speaker_1-kaldi-words', 'speaker_2-kaldi-words']


In [13]:
cd /srv/scratch/z5313567

/srv/scratch/z5313567
